In [21]:
import pandas as pd
import numpy as np 
import folium 
from folium.plugins import HeatMap


In [22]:
#Creacion de un diccionario con los estados, los usaremos para graficarlos al final.
estados_coords = {
    'Aguascalientes': (21.8853, -102.2916),
    'Baja California': (30.8406, -115.2838),
    'Baja California Sur': (26.0444, -111.6661),
    'Campeche': (19.8301, -90.5349),
    'Chiapas': (16.7569, -93.1292),
    'Chihuahua': (28.6330, -106.0691),
    'Coahuila': (27.0587, -101.7068),
    'Colima': (19.2452, -103.7241),
    'Durango': (24.0277, -104.6532),
    'Estado de México': (19.3548, -99.6304),
    'Guanajuato': (21.0190, -101.2574),
    'Guerrero': (17.4392, -99.5451),
    'Hidalgo': (20.0911, -98.7624),
    'Jalisco': (20.6595, -103.3494),
    'Michoacán': (19.5665, -101.7068),
    'Morelos': (18.6813, -99.1013),
    'Nayarit': (21.7514, -104.8455),
    'Nuevo León': (25.5922, -99.9758),
    'Oaxaca': (17.0732, -96.7266),
    'Puebla': (19.0414, -98.2063),
    'Querétaro': (20.5888, -100.3899),
    'Quintana Roo': (19.1817, -88.4791),
    'San Luis Potosí': (22.1565, -100.9855),
    'Sinaloa': (25.1721, -107.4795),
    'Sonora': (29.2972, -110.3309),
    'Tabasco': (17.8409, -92.6189),
    'Tamaulipas': (24.2669, -98.8363),
    'Tlaxcala': (19.3139, -98.2404),
    'Veracruz': (19.1738, -96.1342),
    'Yucatán': (20.7099, -89.0943),
    'Zacatecas': (22.7709, -102.5832)
}

In [23]:
class AnalizadorDeDatos:
    def __init__(self, file_name):
        self.file_name = file_name
        self.data =  None # Por que iniciamos un data con nada?
        self.estados_coords = estados_coords
    
    def open_file(self):
        try:
            if self.file_name.endswith('.csv'):
                self.data = pd.read_csv(self.file_name)
                print(f"Your file was oppend sucesfully")
            elif self.file_name.endswith('.xls') or self.file_name.endswith('.xlsx'):
                self.data = pd.read_excel(self.file_name)
                print(f'Your file was oppened sucessfully')
            else:
                print(f"This file is not valid, please introduce one of the next: csv / .xlsx / .xls ")
        except Exception as e:
            print(f'Ocurrio un eror al abrir el erchivo, codigo de error : {e}')
    
    def show_rows(self, rows = 10):
        if self.data is not None:
            return self.data.head(rows)
        else:
            print(f'Is not file upload yet')
    
    def show_info(self):
        if self.data is not None:
            return self.data.info()
        else:
            print(f'Is not file upload yet')
    
    def show_describe(self):
        if self.data is not None:
            return self.data.describe()
        else:
            print(f"There is not a file open yet")
    
    #Mas bien es para renombrar la columna, hay que realizar esa modificacion
    def extraer_calificacion(self, columna): #<- columna sera el nombre de la columna que queremos revisar
        #Primero nos aseguraremos que el data-frame tiene datos y que la comlumna existe
        if self.data is not None and columna in self.data.columns:
            
            # Crear nuevas columnas para almacenar la calificación extraída y su valor numérico
            columna_calificacion = f'{columna}_calificación'
            columna_calificacion_num = f'{columna}_calificación_num'
            
            self.data[columna_calificacion] = self.data[columna].str.extract(r'(\d+/10)')
            self.data[columna_calificacion_num] = self.data[columna_calificacion].str.split('/').str[0].astype(float)
            print(f"Calificación extraída con éxito de la columna {columna}.")
        else:
            print(f"Asegúrate de que el archivo haya sido cargado y contenga una columna '{columna}'.")
    
    def mostrar_nombres_columnas(self):
        cols = []
        if self.data is not None:
            print("Nombres de las columnas:")
            for col in self.data.columns:
                print(col)
                print('///')
                cols.append(col)
        else:
            print("No hay datos cargados.")   
        return cols # Retornamos una lista de todas las columnas que tenemos.
    
    def renombrar_columnas(self, nombres_actuales, nuevos_nombres):
        
    
    
        if self.data is not None:
            if len(nombres_actuales) != len(nuevos_nombres):
                print("Las listas de nombres actuales y nuevos nombres deben tener la misma longitud.")
                return
            
            mapeo_nombres = dict(zip(nombres_actuales, nuevos_nombres))
            
            self.data.rename(columns=mapeo_nombres, inplace=True)
            print("Las columnas han sido renombradas con éxito.")
        else:
            print("Asegúrate de que el archivo haya sido cargado.")

    def extraer_primer_digito_calificacion(self, columna_origen, nueva_columna):
        if self.data is not None and columna_origen in self.data.columns:
            # Extraer el primer dígito antes del '/'.
            self.data[nueva_columna] = self.data[columna_origen].str.extract(r'(\d)/')
            
            # Llenar los valores NaN con "0". sasa
            self.data[nueva_columna].fillna("0", inplace=True)
            
            # Eliminar la columna original.
            self.data.drop(columns=columna_origen, inplace=True)
            
            print(f"Primer dígito de calificación extraído con éxito en la columna '{nueva_columna}'.")
        else:
            print("Asegúrate de que el archivo haya sido cargado y contenga la columna especificada.")


    def add_states(self, states = estados_coords):
        ####
        self.data['estado'] = np.random.choice(list(estados_coords.keys()), size=len(self.data))
        ###
        self.data["Cordenates"] = self.data["estado"].map(estados_coords)


    def show_map(self):
        m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

        for estado, coords in self.estados_coords.items():
            count = self.data[self.data['estado'] == estado].shape[0]
            folium.Circle(
                location=coords,
                radius=count*10,
                color='blue',
                fill=True,
                fill_color='blue'
            ).add_to(m)

        return m
    

    
    from folium.plugins import HeatMap



    def show_heatmap(self):
        # Crea un mapa centrado en México.
        m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)
        
        # Esta lista almacenará las coordenadas y pesos para el mapa de calor.
        heat_data = []
        
        # Para cada estado, calcula la suma de calificaciones.
        for estado, coords in self.estados_coords.items():
            total_calificacion = self.data[self.data['estado'] == estado]['Calificacion_1_Digito'].sum()
            # Multiplica las coordenadas por el total de calificación para ponderarlas.
            for _ in range(int(total_calificacion)):
                heat_data.append(coords)
        
        # Añade el HeatMap al mapa.
        HeatMap(heat_data).add_to(m)

        return m

    

        
            
#Es diferente usar entre !=  y is not:
#!= compara los valores de dos objetos.
#is y is not verifican la identidad de los objetos, es decir, si dos variables apuntan al mismo objeto en memoria.


In [24]:
Barbie_file = AnalizadorDeDatos('imdb_barbie_Uncleaned.csv')
Oppenhim_File = AnalizadorDeDatos('imdb_oppenhimmer_Uncleaned.csv')

Barbie_file.open_file()
Oppenhim_File.open_file()


Your file was oppend sucesfully
Your file was oppend sucesfully


La calificación ("6/10").
"El título de la revisión ("Creative but preachy and confusing").
El nombre del revisor ("Imshortok0623").
La fecha de la revisión ("July 2023").
El contenido detallado de la revisión.
La utilidad de la revisión ("1,007 out of 1,363 found this helpful").

In [25]:
#Barbie_file.show_rows(3)
#Barbie_file.show_describe()
Barbie_file.show_info()
#Barbie_file.show_rows(4)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13097 entries, 0 to 13096
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [26]:
name_column = Barbie_file.mostrar_nombres_columnas()
new_names = ['Clasificacion 6/10']
Barbie_file.renombrar_columnas(name_column, new_names)

Nombres de las columnas:
6/10
Creative but preachy and confusing
Imshortok0623 July 2023
I'm just going to get straight to the point: I liked some aspects of this movie and disliked other parts. Cinematography: awesome. Choreographed musical numbers: hilarious. Comedy: good. Margot Robbie: great. Ryan Gosling: awesome, and arguably the best part of the movie. The scenery and props really did help create the world. The storyline started off very intriguing. However, when the movie got to the meat of the plot and the central conflict, its message became confusing. It's like they wanted to make Ken the bad guy yet also sympathetic, which is not impossible or unheard of, but the way the writers did this was ineffective. Also, the resolution didn't make sense based on the message and events that led up to it. Lastly, I really hate it when movies try to empower women by dumbing down the men. I get that in a Barbie world, Ken is an afterthought, but in the real world all men were also dumb. T

In [27]:
Barbie_file.show_info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13097 entries, 0 to 13096
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Clasificacion 6/10  13097 non-null  object
dtypes: object(1)
memory usage: 102.4+ KB


In [28]:
#Barbie_file.show_describe()
print(Barbie_file.show_rows(4))
Barbie_file.mostrar_nombres_columnas()

                                  Clasificacion 6/10
0  6/10\nBeautiful film, but so preachy\nLoveofLe...
1  6/10\nClever and fun beginning turns into anot...
2  8/10\nIt was depressing\ncoxaneesa24 July 2023...
3  9/10\nA Technicolor Dream\nheatherhilgers24 Ju...
Nombres de las columnas:
Clasificacion 6/10
///


['Clasificacion 6/10']

In [29]:
Barbie_file.extraer_primer_digito_calificacion('Clasificacion 6/10', 'Calificacion_1_Digito')

Primer dígito de calificación extraído con éxito en la columna 'Calificacion_1_Digito'.


In [30]:
Barbie_file.show_rows(10)


,Calificacion_1_Digito
0,6
1,6
2,8
3,9
4,7
5,8
6,6
7,8
8,6
9,8


In [31]:
Barbie_file.add_states(estados_coords)
Barbie_file.show_rows(10)


,Calificacion_1_Digito,estado,Cordenates
0,6,Nayarit,"(21.7514, -104.8455)"
1,6,Tamaulipas,"(24.2669, -98.8363)"
2,8,Nayarit,"(21.7514, -104.8455)"
3,9,Hidalgo,"(20.0911, -98.7624)"
4,7,Tlaxcala,"(19.3139, -98.2404)"
5,8,Hidalgo,"(20.0911, -98.7624)"
6,6,Chihuahua,"(28.633, -106.0691)"
7,8,Michoacán,"(19.5665, -101.7068)"
8,6,Campeche,"(19.8301, -90.5349)"
9,8,Quintana Roo,"(19.1817, -88.4791)"


In [32]:
Barbie_file.show_map()

In [33]:
Barbie_file.show_heatmap()

KeyboardInterrupt: 